In [2]:
import pandas as pd
import numpy as np
from collections import Counter
import math

In [3]:
with open("../../Project 2/CS205_SP_2022_SMALLtestdata__44.txt", "r") as file:
    data= file.readlines()


In [4]:
dt_row=[]
for line in data:
    row=[]
    for col in line.split():
        row.append(float(col))
    dt_row.append(row)





In [30]:
df= pd.DataFrame(dt_row, columns=['target', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10'])

### Leave out one cross validation

In [31]:
cols= list(df.columns)
cols1= cols[1:]
cols1.append(cols[0])
df= df[cols1]
df['target']= df['target'].astype('int')

#all_vals = df[list(df.columns)[:-1]].values
#all_labels = df[list(df.columns)[-1]].values
#all_vals.shape, all_labels.shape

In [54]:
def compute_dist(test_pt, train_pt):
    dist_mat= np.round(math.sqrt(np.sum(np.abs(train_pt-test_pt)**2)), 3)
    return dist_mat
    
def knn_classifier(test_data, training_data, training_labels, K=1):
    pred_labels = np.zeros(len(test_data))
    #print("pred_labels: ", pred_labels)

    all_dists = np.zeros(len(training_data))
        #print("all_dists: ", all_dists.shape)
    count = 0
    for train_pt in training_data: #for every row in train data
        all_dists[count] = compute_dist(test_data, train_pt)
        #print(all_dists[count])
        count = count+1
    
    sorted_indices = np.argsort(all_dists)
    pred_labels = training_labels[sorted_indices[0]]
    #print(len(pred_labels))
    return pred_labels

In [115]:
# split data based on Leave out one cross validation
def loov(features_list):
    #print("features_list: ", features_list)
    
    all_accuracy= []
    for i in range(len(df)):
        #if i%50==0:
        #    print(i)
        #train-test split
        test_df= df.iloc[[i, ]]
        train_df= pd.concat([df.iloc[:i, ], df.iloc[i+1: ,]])
        X_train, X_test, y_train, y_test= train_df.iloc[:, features_list] , test_df.iloc[:, features_list], train_df.iloc[:, -1], test_df.iloc[:, -1]
        #print("X_train: ", X_train.shape, X_test.shape, y_train.shape, y_test.shape)
        #print(X_train)
        #print(X_test)
        pred_labels = knn_classifier(X_test.values, X_train.values, y_train.values, 1)
        accuracy = sum(y_test.values == pred_labels)
        #print(pred_labels, y_test.values, accuracy, len(pred_labels))
        all_accuracy.append(accuracy)
    return np.round(np.array(all_accuracy).mean(),3)
    

In [119]:
#backward

best_acc= 0
best_features= []

#converting to integer for indexing
org_cols_list= [int(col)-1 for col in list(df.columns)[:-1]]
feat_list=org_cols_list.copy()

for itr in org_cols_list: #1->2->3->4 (how many features)
    print(f"Iteration: {itr} :: current feature list: {feat_list}")
    if itr==0:
        best_acc= loov(org_cols_list)
        best_features= org_cols_list.copy()
    else:
        features_acc=[0 for col in org_cols_list]
        #remove one feature from feat_list
        for idx in range(len(feat_list)): #what features
            copy_feat_list= feat_list[:idx]+feat_list[idx+1:]#.remove(feat)
            acc= loov(copy_feat_list) #interpret as: all exisiting features w/o feat
            features_acc[feat_list[idx]]= acc
    
        #calc best accuracy: eleimate feature that had least drop in accuracy: meaning less important feature. If biggest drop in accuracy: more imp feature
        features_acc_ind= np.argsort(features_acc)
        #print("features_acc: ", features_acc)
        print(f"Worst feature: {features_acc_ind[-1]} with accuracy {features_acc[features_acc_ind[-1]]}")
        
        feat_list.remove(features_acc_ind[-1]) #remove worst acc
        #print("feat_list: ", feat_list)

        if features_acc[features_acc_ind[-1]] > best_acc:
            best_features.remove(features_acc_ind[-1])
            best_acc= features_acc[features_acc_ind[-1]]
        
        
    print(f"Best_features till now: {best_features}, Best_accuracy till now: {best_acc} \n")

Iteration: 0 :: current feature list: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Best_features till now: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], Best_accuracy till now: 0.717 

Iteration: 1 :: current feature list: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Worst feature:  8
Best_features till now: [0, 1, 2, 3, 4, 5, 6, 7, 9], Best_accuracy till now: 0.747 

Iteration: 2 :: current feature list: [0, 1, 2, 3, 4, 5, 6, 7, 9]
Worst feature:  0
Best_features till now: [1, 2, 3, 4, 5, 6, 7, 9], Best_accuracy till now: 0.78 

Iteration: 3 :: current feature list: [1, 2, 3, 4, 5, 6, 7, 9]
Worst feature:  5
Best_features till now: [1, 2, 3, 4, 6, 7, 9], Best_accuracy till now: 0.813 

Iteration: 4 :: current feature list: [1, 2, 3, 4, 6, 7, 9]
Worst feature:  9
Best_features till now: [1, 2, 3, 4, 6, 7], Best_accuracy till now: 0.847 

Iteration: 5 :: current feature list: [1, 2, 3, 4, 6, 7]
Worst feature:  6
Best_features till now: [1, 2, 3, 4, 7], Best_accuracy till now: 0.873 

Iteration: 6 :: current feature list: [1, 2,

In [120]:
#feature selection
#baseline accuracy: 
freq_count= dict(df['target'].value_counts(ascending=False))
#for every test point: assign label as the most frequent class in train data
base_line_acc= np.round(list(freq_count.values())[0]/len(df), 3)
print("base_line_acc: ", base_line_acc)

#converting to integer for indexing
org_cols_list= [int(col)-1 for col in list(df.columns)[:-1]]

print("org_feat_list: ", org_cols_list)
best_acc= 0
best_features= []

feat_list=[]

#forward: keep best
for itr in org_cols_list: #1->2->3->4 (how many features)
    print(f"Iteration: {itr} :: current feature list: {feat_list}")
    features_acc=[0 for col in org_cols_list]
    for feat in org_cols_list: #what features
        if feat not in feat_list:
            #print(int(feat))
            acc= loov(feat_list+[feat])
            features_acc[feat]= acc
    
    #calc best accuracy
    features_acc_ind= np.argsort(features_acc)
    print(f"Best feature: {features_acc_ind[-1]} with accuracy {features_acc[features_acc_ind[-1]]}")
    #print("features_acc: ", features_acc)

    feat_list.append(org_cols_list[features_acc_ind[-1]])
    #print("feat_list: ", feat_list)
    if features_acc[features_acc_ind[-1]] > best_acc:
        best_features.append(org_cols_list[features_acc_ind[-1]])
        best_acc= features_acc[features_acc_ind[-1]]
        
        
    print(f"Best_features till now: {best_features}, Best_accuracy till now: {best_acc} \n")
#backward: remove worst



base_line_acc:  0.813
org_feat_list:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Iteration: 0 :: current feature list: []
Best feature:  3
Best_features till now: [3], Best_accuracy till now: 0.827 

Iteration: 1 :: current feature list: [3]
Best feature:  1
Best_features till now: [3, 1], Best_accuracy till now: 0.957 

Iteration: 2 :: current feature list: [3, 1]
Best feature:  8
Best_features till now: [3, 1], Best_accuracy till now: 0.957 

Iteration: 3 :: current feature list: [3, 1, 8]
Best feature:  7
Best_features till now: [3, 1], Best_accuracy till now: 0.957 

Iteration: 4 :: current feature list: [3, 1, 8, 7]
Best feature:  4
Best_features till now: [3, 1], Best_accuracy till now: 0.957 

Iteration: 5 :: current feature list: [3, 1, 8, 7, 4]
Best feature:  9
Best_features till now: [3, 1], Best_accuracy till now: 0.957 

Iteration: 6 :: current feature list: [3, 1, 8, 7, 4, 9]
Best feature:  5
Best_features till now: [3, 1], Best_accuracy till now: 0.957 

Iteration: 7 :: current featu